In [1]:
# (Optional) Set the environment variable 'HF_ENDPOINT' to use the specified endpoint URL.
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
from utils import chatgpt,CronQuestions,calculate_accuracy
from prompt import DECISION_INSTRUCTION
import tqdm
import time
from cronquestions import get_actions,Inference
import pickle

# Start from new

In [3]:
# start from new
cronq = CronQuestions()
inference = Inference(cronq.kg,cronq.id2text,cronq.rel2text,history=[])


# or Load history_dict / inference

In [7]:
# start from history_dict (use method instruct)
with open('./history_dict_sample.pkl','rb') as f:
    history_dict = pickle.load(f)
inference = Inference(cronq.kg,cronq.id2text,cronq.rel2text,history=history_dict,n_clusters=10)

In [ ]:
## start from the saved inference
# with open('../pkl_data/cronq/inference_cronq_0_200_without_examples_with_memory_round_1(10cluster).pkl','rb') as f:
#     inference = pickle.load(f)



# Inference step

In [8]:
def decision(qid):
    inference.reset()
    attemp = 0
    entities = None
    max_attemp = 5
    q,head_text,rel_text,tail_text,t,event_text,answers,tq = cronq[qid]
    print('Question {}:'.format(qid),q)
    hit = False
    while True:
        attemp+=1
        choose = get_actions(head_text,rel_text,tail_text,t,event_text,entities)
        # print('All generated actions:',len(choose))
        choose = inference.action_filter(choose)
        if len(choose) == 0:
            print('Unable to get answer!')
            inference.history.append([['Result: Unable to get answer!']])
            break
        print('-'*100)
        print('Step:',inference.tick)
        print('Filtered generated actions:',len(choose))
        choose_text = '\n'.join(choose)
        print(choose_text)

        history = ''
        for i in inference.history:
            history+= '\n'+ ' '.join(i)
        prompt = DECISION_INSTRUCTION.replace('{question}',q)
        prompt = prompt.replace('{actions}',choose_text)
        prompt = prompt.replace('{history}',history if len(history)>0 else 'None')
        # add method instruction
        # prompt = prompt.replace('{methodology}',inference.memory.get_method_instruction(tq))
        append = ' What should we do first?' if history == '' else ' What should we do next?'
        print('-'*100)
        print("History Decision:",history)

        print("LLM's Decision:")
        response = chatgpt(prompt)

        print(response)
        kg_res,entities,end = inference.take_action(response)  
        if end:
            print('The predicted answer is:',kg_res[0])            
            ans = cronq.question[qid]['answers']
            hit = inference.check_correctness(ans,kg_res[0])

            if hit:
                print('Correct!')
                inference.history.append([['Result: Correct!']])
                break
            else:
                print('Wrong!')
                inference.history.append([['Result: Wrong!']])
                break
                
        if attemp>=max_attemp:
            print('Unable to get answer!')
            inference.history.append([['Result: Unable to get answer!']])
            break
    his = inference.get_history()
    inference.reset()
    return hit,his

In [9]:
history_dict = {}
start_id = 10000
end_id = 10005
for qid in tqdm.tqdm(range(start_id,end_id)):
    hit,his = decision(qid)
    history_dict[qid] = {'question':cronq.question[qid],'process':his,'result':hit}
    # time.sleep(2)


  0%|          | 0/5 [00:00<?, ?it/s]

Question 10000: What year did Georges Millot get honorary doctorate
----------------------------------------------------------------------------------------------------
Step: 0
Filtered generated actions: 16
$get_tail_entity(Georges Millot,member of,no time constraints)$
$get_head_entity(doctor honoris causa,member of,no time constraints)$
$get_head_entity(doctor honoris causa,position held,no time constraints)$
$get_head_entity(doctor honoris causa,honorific suffix,no time constraints)$
$get_time(Georges Millot,award received,doctor honoris causa)$
$get_head_entity(doctor honoris causa,affiliation,no time constraints)$
$get_tail_entity(Georges Millot,educated at,no time constraints)$
$get_tail_entity(Georges Millot,affiliation,no time constraints)$
$get_head_entity(doctor honoris causa,academic degree,no time constraints)$
$get_head_entity(doctor honoris causa,occupation,no time constraints)$
$get_head_entity(doctor honoris causa,award received,no time constraints)$
$get_head_entity(d

 20%|██        | 1/5 [00:05<00:20,  5.13s/it]

Action:
$answer(1981)$

Reason:
Based on the previous actions, we have already retrieved the years when Georges Millot received the honorary doctorate, which included 1979, 1981, and 1991. Since the question specifically asks for the year he received the honorary doctorate, and 1981 is one of the years listed, it is a correct answer.
The predicted answer is: 1981
Correct!
Question 10001: What was the team Scott Parker played in 1998?
----------------------------------------------------------------------------------------------------
Step: 0
Filtered generated actions: 2
$get_tail_entity(Scott Parker,member of sports team,no time constraints)$
$get_tail_entity(Scott Parker,member of sports team,1998)$
----------------------------------------------------------------------------------------------------
History Decision: 
LLM's Decision:
Action:
$get_tail_entity(Scott Parker,member of sports team,1998)$

Reason:
This action is specifically targeting the relationship of Scott Parker as a me

 40%|████      | 2/5 [00:08<00:12,  4.31s/it]

...

Action:
$answer(Carmelita Geraghty)$

Reason:
The previous action identified that Carey Wilson's spouse was Carmelita Geraghty, and since the question asks for the first person Carey Wilson married, this is the correct answer.
The predicted answer is: Carmelita Geraghty
Correct!
Question 10003: What was the final award Isadore Manuel Singer received?
----------------------------------------------------------------------------------------------------
Step: 0
Filtered generated actions: 1
$get_tail_entity(Isadore Singer,award received,no time constraints)$
----------------------------------------------------------------------------------------------------
History Decision: 
LLM's Decision:
Action:
$get_tail_entity(Isadore Singer,award received,no time constraints)$

Reason:
This action is appropriate because it will help identify the specific awards that Isadore Singer received, without any time constraints, allowing us to find the final award he received.
-------------------------------

 80%|████████  | 4/5 [00:17<00:04,  4.29s/it]

Action:
$answer(Honorary doctoral degree of the Pierre and Marie Curie University)$

Reason:
The previous actions have already identified that the last award received by Isadore Manuel Singer was the Honorary doctoral degree of the Pierre and Marie Curie University in 2005. Since this is the final award he received, it directly answers the question.
The predicted answer is: Honorary doctoral degree of the Pierre and Marie Curie University
Correct!
Question 10004: What is the last time that Elliott Frank Johnston was Justice of the Supreme Court of South Australia
----------------------------------------------------------------------------------------------------
Step: 0
Filtered generated actions: 5
$get_tail_entity(Elliott Johnston,position held,no time constraints)$
$get_time(Elliott Johnston,position held,Justice of the South Australian Supreme Court)$
$get_head_entity(Justice of the South Australian Supreme Court,position held,no time constraints)$
$get_head_entity(Justice of the S

100%|██████████| 5/5 [00:21<00:00,  4.21s/it]

Action:
$answer(1988)$

Reason:
The previous action retrieved the time period during which Elliott Frank Johnston held the position of Justice of the Supreme Court of South Australia, which was from 1983 to 1988. Since the question asks for the last time he held that position, the correct answer is 1988.
The predicted answer is: 1988
Correct!


# Save the history_dict 

In [10]:
with open('./test_history_dict.pkl','wb') as f:
    pickle.dump(history_dict,f)

# Evaluation

In [8]:
calculate_accuracy(history_dict)

simple_entity 	 0.9 63
time_join 	 0.09 23
first_last 	 0.74 70
simple_time 	 0.9 30
before_after 	 0.0 12
Overall Accuracy: 0.696969696969697


0.696969696969697